In [1]:
#kernel: tf-gpu
import DataPrep
import error_evaluation
import pandas as pd
import numpy as np


Using TensorFlow backend.


In [2]:
#dp = DataPrep.DataPrep()
#X_Above_test, X_Above_train, y_Above_test, y_Above_train, X_Near_test, X_Near_train, y_Near_test, y_Near_train, X_Below_test, X_Below_train, y_Below_test, y_Below_train = dp.prep_day1_danger_train_test(input_file='E:/Data/OAPMLData/V1.1CIAC_UAC_NWAC_FeaturesWithLabels30Days20131201To20180430.csv', n_rows=10, ignore_extreme=True, only_critical_points=False, oversample=False, only_precise_points = False, label_critical_points=True)


In [3]:
X_Above_test = pd.read_pickle('E:/Temp/X_test.pkl')
X_Above_train = pd.read_pickle('E:/Temp/X_train.pkl')

y_Above_test = pd.read_pickle('E:/Temp/y_test.pkl')
y_Above_train = pd.read_pickle('E:/Temp/y_train.pkl')

In [4]:
y_Above_test.head()

,o_Day1DangerAboveTreeline,o_Cornices_Likelihood,o_Glide_Likelihood,o_LooseDry_Likelihood,o_LooseWet_Likelihood,o_PersistentSlab_Likelihood,o_DeepPersistentSlab_Likelihood,o_StormSlabs_Likelihood,o_WetSlabs_Likelihood,o_WindSlab_Likelihood,Lat,Lon,UnifiedRegion,Date,Season,IsCriticalPoint
76226,0,no-data,no-data,no-data,no-data,no-data,no-data,no-data,no-data,no-data,37.044847,-105.433882,Sangre de Cristo,2017-03-28,16-17,False
77900,1,no-data,no-data,no-data,1-possible,no-data,no-data,1-possible,no-data,no-data,37.044847,-105.433882,Sangre de Cristo,2017-03-29,16-17,False
79547,0,no-data,no-data,no-data,1-possible,no-data,no-data,0-unlikely,no-data,no-data,37.044847,-105.433882,Sangre de Cristo,2017-03-30,16-17,False
81182,0,no-data,no-data,no-data,no-data,no-data,no-data,0-unlikely,no-data,no-data,37.044847,-105.433882,Sangre de Cristo,2017-03-31,16-17,False
82856,2,no-data,no-data,no-data,no-data,no-data,no-data,2-likely,no-data,no-data,37.044847,-105.433882,Sangre de Cristo,2017-04-01,16-17,False


In [5]:
columns = pd.read_csv('E:/Temp/top1000tsfreshfeatures.csv', header=None)

In [6]:
files_X = ['df_rolled_x_13-14.par','df_rolled_x_14-15.par','df_rolled_x_15-16.par','df_rolled_x_16-17.par','df_rolled_x_17-18.par']
files_y = ['df_rolled_y_13-14.pkl','df_rolled_y_14-15.pkl','df_rolled_y_15-16.pkl','df_rolled_y_16-17.pkl','df_rolled_y_17-18.pkl']


In [7]:
from imblearn.under_sampling import RandomUnderSampler
import gc
rus = RandomUnderSampler(random_state=0)
#X_resampled, y_resampled = rus.fit_resample(ext_df, y_df)
resample = False
dfs_X = []
dfs_y = []
#read all, the merge later will take care of the train test split
for f_i in range(0,len(files_X)):
    path = "E:/Temp/" + files_X[f_i]

    with open(path, 'rb') as f:
        tmp_X = pd.read_parquet(f, columns=columns[0].values)

    tmp_y = pd.read_pickle("E:/Temp/" + files_y[f_i])
    if resample:
        tmp_X_resampled, tmp_y_resampled = rus.fit_resample(tmp_X, tmp_y)
        dfs_X.append(pd.DataFrame(tmp_X_resampled, columns=columns[0].values, index=tmp_X.index[rus.sample_indices_]))
        dfs_y.append(pd.Series(tmp_y_resampled, index=tmp_y.index[rus.sample_indices_]))
        del tmp_X_resampled
        del tmp_y_resampled
        
    else:
        dfs_X.append(pd.DataFrame(tmp_X, columns=columns[0].values))
        dfs_y.append(pd.Series(tmp_y))
    del tmp_X
    del tmp_y
    gc.collect()

In [8]:
dfs_y[3].value_counts()

Moderate        45445
Low             19658
Considerable    19448
High             5894
Name: o_Day1DangerAboveTreeline, dtype: int64

In [9]:
dfs_X_all = pd.concat(dfs_X)
dfs_y_all = pd.concat(dfs_y)

In [10]:
dfs_X_all.shape

(429966, 1000)

In [11]:
dfs_y_all.value_counts()

Moderate        214288
Low             106273
Considerable     89624
High             19781
Name: o_Day1DangerAboveTreeline, dtype: int64

In [12]:
y_Above_test.head()

,o_Day1DangerAboveTreeline,o_Cornices_Likelihood,o_Glide_Likelihood,o_LooseDry_Likelihood,o_LooseWet_Likelihood,o_PersistentSlab_Likelihood,o_DeepPersistentSlab_Likelihood,o_StormSlabs_Likelihood,o_WetSlabs_Likelihood,o_WindSlab_Likelihood,Lat,Lon,UnifiedRegion,Date,Season,IsCriticalPoint
76226,0,no-data,no-data,no-data,no-data,no-data,no-data,no-data,no-data,no-data,37.044847,-105.433882,Sangre de Cristo,2017-03-28,16-17,False
77900,1,no-data,no-data,no-data,1-possible,no-data,no-data,1-possible,no-data,no-data,37.044847,-105.433882,Sangre de Cristo,2017-03-29,16-17,False
79547,0,no-data,no-data,no-data,1-possible,no-data,no-data,0-unlikely,no-data,no-data,37.044847,-105.433882,Sangre de Cristo,2017-03-30,16-17,False
81182,0,no-data,no-data,no-data,no-data,no-data,no-data,0-unlikely,no-data,no-data,37.044847,-105.433882,Sangre de Cristo,2017-03-31,16-17,False
82856,2,no-data,no-data,no-data,no-data,no-data,no-data,2-likely,no-data,no-data,37.044847,-105.433882,Sangre de Cristo,2017-04-01,16-17,False


In [13]:
y_Above_test.index = y_Above_test['Date'].astype(str) + '-' + (y_Above_test['Lat']*100000).astype(int).apply(str) + (y_Above_test['Lon']*-100000).astype(int).apply(str) + y_Above_test['Season']

In [14]:
y_Above_train.index = y_Above_train['Date'].astype(str) + '-' + (y_Above_train['Lat']*100000).astype(int).apply(str) + (y_Above_train['Lon']*-100000).astype(int).apply(str) + y_Above_train['Season']

In [15]:
X_Above_test.index = y_Above_test.index

In [16]:
X_Above_train.index = y_Above_train.index

In [17]:
X_Above_train.head()

,n_f_APCPsurface1HourForecast,n_f_10mWindSpeed1HourForecast,n_f_APCPsurface2HourForecast,n_f_10mWindSpeed2HourForecast,n_f_APCPsurface3HourForecast,n_f_10mWindSpeed3HourForecast,n_f_APCPsurface4HourForecast,n_f_10mWindSpeed4HourForecast,n_f_APCPsurface5HourForecast,n_f_10mWindSpeed5HourForecast,...,c_r_Prev3DayFreezeThawLikeliness29InPast,c_r_Prev7DayFreezeThawLikeliness29InPast,c_r_Prev3DayWindSlabLikeliness29InPast,c_r_Prev7DayWindSlabLikeliness29InPast,c_r_Prev3DayFreezeThawLikeliness30InPast,c_r_Prev7DayFreezeThawLikeliness30InPast,c_r_Prev3DayWindSlabLikeliness30InPast,c_r_Prev7DayWindSlabLikeliness30InPast,c_IsCoastalSnowpack,c_IsContenentalSnowpack
2014-03-08-37044841054338813-14,0.125,4.425314,0.25,3.348404,0.375,2.863331,0.75,2.640573,2.375,4.330175,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2014-03-09-37044841054338813-14,0.000,3.700787,0.00,2.219529,0.000,0.928360,0.00,0.602276,0.000,0.941570,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
2014-03-10-37044841054338813-14,0.000,2.356316,0.00,0.973392,0.000,1.061180,0.00,1.123587,0.000,1.227528,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
2014-03-11-37044841054338813-14,0.000,5.312415,0.00,4.078613,0.000,4.566241,0.00,5.146346,0.000,6.150001,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2014-03-12-37044841054338813-14,0.000,6.636964,0.00,8.509737,0.000,8.911512,0.00,8.556581,0.000,7.402133,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0


In [18]:
X_Above_test_merged = X_Above_test.merge(dfs_X_all, how='inner', left_index=True, right_index=True)

In [19]:
X_Above_train_merged = X_Above_train.merge(dfs_X_all, how='inner', left_index=True, right_index=True)

In [20]:
X_Above_test_merged.head()

,n_f_APCPsurface1HourForecast,n_f_10mWindSpeed1HourForecast,n_f_APCPsurface2HourForecast,n_f_10mWindSpeed2HourForecast,n_f_APCPsurface3HourForecast,n_f_10mWindSpeed3HourForecast,n_f_APCPsurface4HourForecast,n_f_10mWindSpeed4HourForecast,n_f_APCPsurface5HourForecast,n_f_10mWindSpeed5HourForecast,...,SNOWDAS_SnowpackAveTemp_k__variance,MinTempTropF__index_mass_quantile__q_0.8,"AvgTempSurfaceF__fft_coefficient__coeff_7__attr_""real""","MaxTempTropF__fft_coefficient__coeff_25__attr_""abs""","MaxTemp80mAboveGroundF__fft_coefficient__coeff_1__attr_""imag""","MaxTemp2mAboveGroundF__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_5__w_10","AvgWindSpeedTrop__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_4__w_10","AvgTempTropF__fft_coefficient__coeff_17__attr_""real""","MaxTempTropF__fft_coefficient__coeff_29__attr_""angle""","MinTemp2mAboveGroundF__fft_coefficient__coeff_12__attr_""angle"""
2017-03-28-37044841054338816-17,0.00,2.830390,0.01,4.166778,0.05,3.875112,0.1,3.175281,0.1,2.613486,...,511.889157,0.811189,41.250832,129.105433,152.441011,92.308584,52.056049,-115.367449,-41.651519,-59.759901
2017-03-29-37044841054338816-17,3.75,0.886337,7.20,1.988536,10.30,3.408619,12.0,4.518070,12.3,5.379788,...,508.550412,0.812500,23.267792,99.203720,166.714356,92.308584,52.056049,-27.052713,-47.032269,-43.063684
2017-03-30-37044841054338816-17,0.00,1.908402,0.00,1.967396,0.00,2.083212,0.0,2.519561,0.0,2.239626,...,505.251948,0.813793,9.875822,61.352369,180.993784,92.308584,52.056049,45.581521,-46.694839,-28.610168
2017-03-31-37044841054338816-17,0.00,3.232829,0.00,3.107298,0.00,3.395886,0.0,4.155732,0.0,4.745046,...,501.993076,0.808219,-0.411190,63.872247,195.377189,92.308584,52.056049,112.797715,-24.377827,-9.206447
2017-04-01-37044841054338816-17,1.05,3.241758,4.12,2.425065,8.69,2.687768,14.0,3.363188,16.1,4.923554,...,498.773117,0.809524,-5.089069,121.540992,209.374272,92.308584,52.056049,155.052106,4.586193,9.093303


In [21]:
X_Above_test.head()

,n_f_APCPsurface1HourForecast,n_f_10mWindSpeed1HourForecast,n_f_APCPsurface2HourForecast,n_f_10mWindSpeed2HourForecast,n_f_APCPsurface3HourForecast,n_f_10mWindSpeed3HourForecast,n_f_APCPsurface4HourForecast,n_f_10mWindSpeed4HourForecast,n_f_APCPsurface5HourForecast,n_f_10mWindSpeed5HourForecast,...,c_r_Prev3DayFreezeThawLikeliness29InPast,c_r_Prev7DayFreezeThawLikeliness29InPast,c_r_Prev3DayWindSlabLikeliness29InPast,c_r_Prev7DayWindSlabLikeliness29InPast,c_r_Prev3DayFreezeThawLikeliness30InPast,c_r_Prev7DayFreezeThawLikeliness30InPast,c_r_Prev3DayWindSlabLikeliness30InPast,c_r_Prev7DayWindSlabLikeliness30InPast,c_IsCoastalSnowpack,c_IsContenentalSnowpack
2017-03-28-37044841054338816-17,0.00,2.830390,0.01,4.166778,0.05,3.875112,0.1,3.175281,0.1,2.613486,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2017-03-29-37044841054338816-17,3.75,0.886337,7.20,1.988536,10.30,3.408619,12.0,4.518070,12.3,5.379788,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2017-03-30-37044841054338816-17,0.00,1.908402,0.00,1.967396,0.00,2.083212,0.0,2.519561,0.0,2.239626,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2017-03-31-37044841054338816-17,0.00,3.232829,0.00,3.107298,0.00,3.395886,0.0,4.155732,0.0,4.745046,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2017-04-01-37044841054338816-17,1.05,3.241758,4.12,2.425065,8.69,2.687768,14.0,3.363188,16.1,4.923554,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0


In [22]:
dfs_X_all.tail()

,APCPSurface__mean_change,"SNOWDAS_SolidPrecip_kgpersquarem__fft_coefficient__coeff_73__attr_""abs""","APCPSurface__change_quantiles__f_agg_""mean""__isabs_False__qh_1.0__ql_0.0","SNOWDAS_SnowDepth_mm__change_quantiles__f_agg_""var""__isabs_False__qh_1.0__ql_0.2","APCPSurface__agg_linear_trend__f_agg_""min""__chunk_len_10__attr_""stderr""","SNOWDAS_SnowDepth_mm__fft_coefficient__coeff_73__attr_""abs""","SNOWDAS_SolidPrecip_kgpersquarem__fft_coefficient__coeff_67__attr_""abs""","SNOWDAS_SnowDepth_mm__fft_coefficient__coeff_62__attr_""abs""","SNOWDAS_SnowDepth_mm__fft_coefficient__coeff_65__attr_""abs""","SNOWDAS_SolidPrecip_kgpersquarem__fft_coefficient__coeff_82__attr_""abs""",...,SNOWDAS_SnowpackAveTemp_k__variance,MinTempTropF__index_mass_quantile__q_0.8,"AvgTempSurfaceF__fft_coefficient__coeff_7__attr_""real""","MaxTempTropF__fft_coefficient__coeff_25__attr_""abs""","MaxTemp80mAboveGroundF__fft_coefficient__coeff_1__attr_""imag""","MaxTemp2mAboveGroundF__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_5__w_10","AvgWindSpeedTrop__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_4__w_10","AvgTempTropF__fft_coefficient__coeff_17__attr_""real""","MaxTempTropF__fft_coefficient__coeff_29__attr_""angle""","MinTemp2mAboveGroundF__fft_coefficient__coeff_12__attr_""angle"""
__fileDate,,,,,,,,,,,,,,,,,,,,,
2018-04-30-48947881216198717-18,0.002483,1162.454257,0.002483,9341.089895,0.172473,705.515026,853.208113,1179.104261,848.181712,0.0,...,487.096563,0.802632,0.346974,159.783472,30.458177,62.008702,139.329270,53.352089,68.412259,-46.814998
2018-04-30-48964691206990017-18,0.006623,682.316592,0.006623,4318.349931,0.081194,764.209414,709.675608,972.416815,766.467075,0.0,...,486.262422,0.802632,9.185321,85.017322,27.560584,58.654324,154.478434,45.297966,77.526352,-34.894118
2018-04-30-48967221214713217-18,0.007450,1370.719699,0.007450,8316.061477,0.171341,749.103374,829.019063,1503.838758,1255.604487,0.0,...,487.096563,0.802632,-10.217703,150.549573,72.588414,59.287360,139.272042,53.082663,70.422492,-55.091909
2018-04-30-48983381205502217-18,0.002483,638.528727,0.002483,3149.658264,0.056686,466.216871,523.965687,514.993179,614.290856,0.0,...,486.406337,0.802632,17.507412,107.511573,9.583826,58.591217,152.377721,45.408494,85.291078,-27.289403
2018-04-30-48986461213227017-18,0.011589,1410.674283,0.011589,6646.262222,0.140097,811.033387,709.532019,1407.345385,1183.476226,0.0,...,487.096563,0.802632,15.758695,145.012498,115.679885,57.402598,146.776169,51.830614,68.217232,-52.517015


In [23]:
pd.DataFrame(dfs_y_all).head()

,o_Day1DangerAboveTreeline
__fileDate,
2013-11-02-37374521069519113-14,Low
2013-11-02-37383891068177913-14,Low
2013-11-02-37393151066836413-14,Low
2013-11-02-37402291080369513-14,Low
2013-11-02-37402301065494713-14,Low


In [24]:
tmp = pd.DataFrame(dfs_y_all)
tmp.columns=['Forecast']
y_Above_train_merged = y_Above_train.merge(tmp, how='inner', left_index=True, right_index=True)

In [25]:
y_Above_test_merged = y_Above_test.merge(tmp, how='inner', left_index=True, right_index=True)

In [26]:
y_Above_test_merged = y_Above_test_merged[~y_Above_test_merged['UnifiedRegion'].isin(['Low East', 'Low West', 'In The Desert', 'In The Lake'])]

In [27]:
X_Above_test_merged = X_Above_test_merged.reindex(y_Above_test_merged.index)

In [28]:
X_Above_test_merged.tail()

,n_f_APCPsurface1HourForecast,n_f_10mWindSpeed1HourForecast,n_f_APCPsurface2HourForecast,n_f_10mWindSpeed2HourForecast,n_f_APCPsurface3HourForecast,n_f_10mWindSpeed3HourForecast,n_f_APCPsurface4HourForecast,n_f_10mWindSpeed4HourForecast,n_f_APCPsurface5HourForecast,n_f_10mWindSpeed5HourForecast,...,SNOWDAS_SnowpackAveTemp_k__variance,MinTempTropF__index_mass_quantile__q_0.8,"AvgTempSurfaceF__fft_coefficient__coeff_7__attr_""real""","MaxTempTropF__fft_coefficient__coeff_25__attr_""abs""","MaxTemp80mAboveGroundF__fft_coefficient__coeff_1__attr_""imag""","MaxTemp2mAboveGroundF__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_5__w_10","AvgWindSpeedTrop__cwt_coefficients__widths_(2, 5, 10, 20)__coeff_4__w_10","AvgTempTropF__fft_coefficient__coeff_17__attr_""real""","MaxTempTropF__fft_coefficient__coeff_29__attr_""angle""","MinTemp2mAboveGroundF__fft_coefficient__coeff_12__attr_""angle"""
2017-03-21-48986461213227016-17,0.00,3.078343,0.0,3.340730,0.00,3.793252,0.0,4.107624,0.0,4.575581,...,534.982777,0.811594,-25.313966,182.990038,87.396589,81.946938,219.383524,-35.583617,57.997216,-125.313518
2017-03-23-48986461213227016-17,0.31,3.762961,0.9,4.208497,2.30,4.167336,5.3,3.750813,6.2,3.681155,...,531.175819,0.812950,-18.783657,190.017072,92.457955,81.946938,219.383524,-58.765195,-149.837657,-103.472148
2017-03-25-48986461213227016-17,0.28,3.757036,0.8,3.922672,1.30,4.298166,1.7,4.495977,2.2,4.521728,...,527.422653,0.807143,-12.421868,185.882842,97.527881,81.946938,219.383524,-83.915018,-61.858026,-80.023703
2017-03-26-48986461213227016-17,0.04,2.083329,0.3,1.702247,0.75,3.513634,0.8,2.617066,0.9,2.858286,...,523.722147,0.808511,-10.898506,159.869556,102.535700,81.946938,219.383524,-119.696785,-28.975763,-63.799600
2017-03-27-48986461213227016-17,0.02,3.271161,0.1,2.939433,0.20,2.729102,0.7,2.779177,1.1,2.930542,...,520.073200,0.809859,-7.193367,120.932863,107.584625,81.946938,219.383524,-136.447494,7.160117,-35.077764


In [29]:
del tmp

In [30]:
y_Above_train_merged['Forecast'].value_counts()

Moderate        115823
Considerable     66205
Low              64371
High             13353
Name: Forecast, dtype: int64

In [31]:
pd.__version__

'0.24.2'

In [32]:
X_Above_train_merged.to_pickle('E:/Temp/combinedtsfreshandseries_X_train.pkl')
X_Above_test_merged.to_pickle('E:\Temp\combinedtsfreshandseries_X_test.pkl')
y_Above_test_merged.to_pickle('E:\Temp\combinedtsfreshandseries_y_test.pkl')
y_Above_train_merged.to_pickle('E:\Temp\combinedtsfreshandseries_y_train.pkl')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
m = BalancedRandomForestClassifier(n_jobs = 8,
                           #oob_score=True,
                           n_estimators=200,
                           max_features="sqrt",
                           min_samples_leaf=400
                           )
m.fit(X_Above_train_merged.values, y_Above_train_merged['Forecast'].values.ravel())
m.score(X_Above_train_merged.values, y_Above_train_merged['Forecast'].values.ravel())

In [ ]:
import error_evaluation
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
preds = m.predict(X_Above_test_merged.values)

error_evaluation.evaluateSingleClassShort(y_Above_test_merged['Forecast'], preds)
cnf_matrix = confusion_matrix(y_Above_test_merged['Forecast'], preds)
plt.figure()
error_evaluation.plot_confusion_matrix(cnf_matrix, classes=['Considerable', 'High', 'Low', 'Moderate'], normalize=True,
                      title='Normalized confusion matrix')

plt.figure()
error_evaluation.plot_confusion_matrix(cnf_matrix, classes=['Considerable', 'High', 'Low', 'Moderate'],
                      title='Confusion matrix, without normalization')

In [ ]:
y_Above_train_merged['Forecast'].value_counts()

In [ ]:
del X_Above_test, X_Above_train, dfs_X_all

In [ ]:
gc.collect()

In [ ]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, ADASYN
#ros = RandomOverSampler(random_state=42)
#ros = SMOTE(random_state=42)
ros = ADASYN(random_state=42, n_jobs=8)
X_Above_train_res, y_Above_train_res = ros.fit_resample(X_Above_train_merged, y_Above_train_merged['Forecast'].values.ravel())


In [ ]:
X_Above_train_res.shape

In [ ]:
y_Above_train_res.shape

In [ ]:
pd.Series(y_Above_train_res).value_counts()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
m = RandomForestClassifier(n_jobs = 8,
                           #oob_score=True,
                           n_estimators=200,
                           max_features="sqrt",
                           min_samples_leaf=4
                           )
m.fit(X_Above_train_res, y_Above_train_res)


In [ ]:
m.score(X_Above_train_res, y_Above_train_res)

In [ ]:
import error_evaluation
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
preds = m.predict(X_Above_test_merged.values)

error_evaluation.evaluateSingleClassShort(y_Above_test_merged['Forecast'], preds)
cnf_matrix = confusion_matrix(y_Above_test_merged['Forecast'], preds)
plt.figure()
error_evaluation.plot_confusion_matrix(cnf_matrix, classes=['Considerable', 'High', 'Low', 'Moderate'], normalize=True,
                      title='Normalized confusion matrix')

plt.figure()
error_evaluation.plot_confusion_matrix(cnf_matrix, classes=['Considerable', 'High', 'Low', 'Moderate'],
                      title='Confusion matrix, without normalization')

In [ ]:
importances = pd.Series(m.feature_importances_)
out = pd.concat([pd.Series(X_Above_train_merged.columns), importances], axis=1)
out.columns=['Feature','Score']
out.sort_values(by=['Score'], ascending=False)[:100]